ChainerMNで[MNIST](https://en.wikipedia.org/wiki/MNIST_database)(手書き数字認識)のサンプルを動かします。<br>
まず、ChainerMNのGitHubからソースコードをダウンロードします。

In [ ]:
curl -L https://github.com/chainer/chainermn/archive/v1.0.0b2.tar.gz | tar zx

chainermn-1.0.0b2というディレクトリが作成されます。

In [ ]:
ls

MNISTのサンプルはchainermn-1.0.0b2/examples/mnist以下にあります。

In [ ]:
ls chainermn-1.0.0b2/examples/mnist

In [ ]:
cat chainermn-1.0.0b2/examples/mnist/train_mnist.py

どのようなオプションがあるか確認してみましょう。

In [ ]:
python chainermn-1.0.0b2/examples/mnist/train_mnist.py -h

"-g"オプションでGPUを使った実行をすることができます。<br>
"-e 数値"オプションでepch数を指定することができます。<br>
"-b 数値"オプションでミニバッチサイズを指定することができます。このミニバッチサイズに並列に実行するプロセス数を掛けたものが全体のバッチサイズとなります。<br>

このtrain_mnist.pyをMPIを使って実行します。<br>
MPIを使って並列にプロセスを実行する場合にはmpirun (またはmpiexec)を使用します。<br>
どのようなオプションがあるか確認しましょう。

In [ ]:
mpirun -h

"-n 数値"オプションで並列に実行するプロセス数を指定することができます。<br>
例えば、AAICは1ノードあたり8GPU搭載しているので、1ノードで実行する場合は"-n 8"とします。
2ノードで実行する場合は16GPU(2ノード×8GPU)使うことになるとで"-n 16"とします。

MPIでPythonプログラムを実行する場合は、<br>
mpirun [MPIのオプション] python [Pythonのプログラム名] [Pythonのプログラムのオプション]<br>
という順で引数を記述します。

In [ ]:
mpirun -np 4 python chainermn-1.0.0b2/examples/mnist/train_mnist.py -g

resultというディレクトリに実行結果が出力されます。

In [ ]:
ls result

cg.dotはDOT言語で記述されたネットワーク構造のファイル、logはJSONで記述された実行時間、エポック数、反復回数、精度などを記述したファイルになります。